::: {.cell .markdown}

In [ ]:
#| echo: false
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/modelisation/0_preprocessing.qmd")

:::

Ce chapitre utilise le jeu de données présenté dans l'[introduction
de cette partie](https://linogaliana-teaching.netlify.app/modelisation/) :
les données de vote aux élections présidentielles américaines de 2020 au niveau des comtés
croisées à des variables socio-démographiques.
Le code de consitution de la base de données
est disponible [sur Github](https://github.com/linogaliana/python-datascientist/blob/master/content/course/modelisation/get_data.py).
L'exercice 1 permet, à ceux qui le désirent, d'essayer de le reconstituer pas à pas. 

Le guide utilisateur de `scikit` est une référence précieuse,
à consulter régulièrement. La partie sur le *preprocessing* est
disponible [ici](https://scikit-learn.org/stable/modules/preprocessing.html).

L'objectif de ce chapitre est de présenter quelques éléments de 
préparation des données. Il s'agit d'une étape fondamentale, à ne
pas négliger. Les modèles reposent sur certaines hypothèses, généralement
relatives à la distribution théorique des variables qui y sont intégrées.

Il est nécessaire de faire correspondre la distribution empirique
à ces hypothèses ce qui implique un travail de restructuration des données.
Celui-ci permettra d'avoir des résultats de modélisation plus pertinents. 
Nous verrons dans le chapitre sur les *pipelines* comment industrialiser
ces étapes de _preprocessing_ afin de se simplifier la vie pour appliquer
un modèle sur un jeu de données différent de celui sur lequel il a été estimé. 

::: {.cell .markdown}

```{=html}
<div class="alert alert-warning" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-lightbulb"></i> scikit-learn </h3>
```


`scikit-learn` est aujourd'hui la librairie de référence dans l'écosystème du
_Machine Learning_. Il s'agit d'une librairie qui, malgré les très nombreuses
méthodes implémentées, présente l'avantage d'être un point d'entrée unifié.
Cet aspect unifié est l'une des raisons du succès précoce de celle-ci. `R` n'a 
bénéficié que plus récemment d'une librairie unifiée,
à savoir [`tidymodels`](https://www.tidymodels.org/).

Une autre raison du succès de `scikit` est son approche opérationnelle: la mise
en production de modèles développés via les _pipelines_ `scikit` est peu coûteuse.
Un [chapitre spécial de ce cours](/pipeline-scikit) est dédié aux _pipelines_.
Avec Romain Avouac, nous proposons un [cours plus avancé](https://ensae-reproductibilite.netlify.app/) 
en dernière année d'ENSAE où nous présentons certains enjeux relatifs
à la mise en production de modèles développés avec `scikit`. 

Le coeur de l'équipe de développement de `scikit-learn` est situé
à l'[Inria](https://www.inria.fr/fr) 🇫🇷. 
Pour découvrir la richesse de l'écosystème `scikit`, il 
est recommandé de suivre le
[`MOOC scikit`](https://www.fun-mooc.fr/fr/cours/machine-learning-python-scikit-learn/),
développé dans le cadre de l'initiative [`Inria Academy`](https://www.inria.fr/fr/mooc-scikit-learn).


```{=html}
</div>
```

:::

# Construction de la base de données

Les sources de données étant diverses, le code qui construit la base finale est directement fourni. Le travail de construction d'une base unique
est un peu fastidieux mais il s'agit d'un bon exercice, que vous pouvez tenter,
pour [réviser `pandas`](#pandas)   :

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 1 : Importer les données des élections US</h3>
```


__Cet exercice est OPTIONNEL__

1. Télécharger et importer le shapefile [depuis ce lien](https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_02_sldl_500k.zip)
2. Exclure les Etats suivants: "02", "69", "66", "78", "60", "72", "15"
3. Importer les résultats des élections depuis [ce lien](https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2020_US_County_Level_Presidential_Results.csv)
4. Importer les bases disponibles sur le site de l'USDA en faisant attention à renommer les variables de code FIPS de manière identique
dans les 4 bases
5. *Merger* ces 4 bases dans une base unique de caractéristiques socioéconomiques
6. *Merger* aux données électorales à partir du code FIPS
7. *Merger* au shapefile à partir du code FIPS. Faire attention aux 0 à gauche dans certains codes. Il est
recommandé d'utiliser la méthode `str.lstrip` pour les retirer
8. Importer les données des élections 2000 à 2016 à partir du [MIT Election Lab](https://electionlab.mit.edu/data)?
Les données peuvent être directement requêtées depuis l'url
<https://dataverse.harvard.edu/api/access/datafile/3641280?gbrecs=false>
9. Créer une variable `share` comptabilisant la part des votes pour chaque candidat. 
Ne garder que les colonnes `"year", "FIPS", "party", "candidatevotes", "share"`
10. Faire une conversion `long` to `wide` avec la méthode `pivot_table` pour garder une ligne
par comté x année avec en colonnes les résultats de chaque candidat dans cet état.
11. Merger à partir du code FIPS au reste de la base. 


```{=html}
</div>
```

:::

Si vous ne faites pas l'exercice 1, pensez à charger les données en executant la fonction `get_data.py` :


In [ ]:
#| echo: true
#| warning: false

#!pip install --upgrade xlrd #colab bug verson xlrd
#!pip install geopandas

import requests

url = 'https://raw.githubusercontent.com/linogaliana/python-datascientist/master/content/course/modelisation/get_data.py'
r = requests.get(url, allow_redirects=True)
open('getdata.py', 'wb').write(r.content)

import getdata
votes = getdata.create_votes_dataframes()

Ce code introduit une base nommée `votes` dans l'environnement. Il s'agit d'une
base rassemblant les différentes sources. Elle a l'aspect
suivant:


In [ ]:
#| echo: false
votes.head(3)

La carte choroplèthe suivante permet de visualiser rapidement les résultats
(l'Alaska et Hawaï ont été exclus). 


In [ ]:
#| warning: false
#| echo: true

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# republican : red, democrat : blue
color_dict = {'republican': '#FF0000', 'democrats': '#0000FF'}

fig, ax = plt.subplots(figsize = (12,12))
grouped = votes.groupby('winner')
for key, group in grouped:
    group.plot(ax=ax, column='winner', label=key, color=color_dict[key])
plt.axis('off')

Les cartes choroplèthes peuvent donner une impression fallacieuse 
ce qui exiplique que 
ce type de carte a servi 
de justification pour contester les résultats du vote.
En effet, un biais
connu des représentations choroplèthes est qu'elles donnent une importance
visuelle excessive aux grands espaces. Or, ceux-ci sont souvent des espaces
peu denses et influencent donc moins la variable d'intérêt (en l'occurrence
le taux de vote en faveur des républicains/démocrates). Une représentation à 
privilégier pour ce type de phénomènes est les
ronds proportionnels (voir @inseeSemiologie, _"Le piège territorial en cartographie"_). 


Le [GIF "Land does not vote, people do"](https://www.core77.com/posts/90771/A-Great-Example-of-Better-Data-Visualization-This-Voting-Map-GIF)
qui avait eu un certain succès en 2020 propose un autre mode de visualisation.
La carte originale a probablement été construite avec `JavaScript`. Cependant,
on dispose avec `Python` de plusieurs outils
pour répliquer, à faible coût, cette carte 
grâce à
l'une des surcouches à JavaScript vue dans la partie [visualisation](#visualisation). 

En l'occurrence, on peut utiliser `plotly` pour tenir compte de la population:


{{< chart data="people_vote" >}}



La Figure a été obtenue avec le code suivant:


In [ ]:
#| warning: false
#| output: false
import plotly
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd


centroids = votes.copy()
centroids.geometry = centroids.centroid
centroids['size'] = centroids['CENSUS_2010_POP'] / 10000  # to get reasonable plotable number

color_dict = {"republican": '#FF0000', 'democrats': '#0000FF'}
centroids["winner"] =  np.where(centroids['votes_gop'] > centroids['votes_dem'], 'republican', 'democrats') 


centroids['lon'] = centroids['geometry'].x
centroids['lat'] = centroids['geometry'].y
centroids = pd.DataFrame(centroids[["county_name",'lon','lat','winner', 'CENSUS_2010_POP',"state_name"]])
groups = centroids.groupby('winner')

df = centroids.copy()

df['color'] = df['winner'].replace(color_dict)
df['size'] = df['CENSUS_2010_POP']/6000
df['text'] = df['CENSUS_2010_POP'].astype(int).apply(lambda x: '<br>Population: {:,} people'.format(x))
df['hover'] = df['county_name'].astype(str) +  df['state_name'].apply(lambda x: ' ({}) '.format(x)) + df['text']

fig_plotly = go.Figure(data=go.Scattergeo(
    locationmode = 'USA-states',
    lon=df["lon"], lat=df["lat"],
    text = df["hover"],
    mode = 'markers',
    marker_color = df["color"],
    marker_size = df['size'],
    hoverinfo="text"
    ))

fig_plotly.update_traces(
  marker = {'opacity': 0.5, 'line_color': 'rgb(40,40,40)', 'line_width': 0.5, 'sizemode': 'area'}
)

fig_plotly.update_layout(
        title_text = "Reproduction of the \"Acres don't vote, people do\" map <br>(Click legend to toggle traces)",
        showlegend = True,
        geo = {"scope": 'usa', "landcolor": 'rgb(217, 217, 217)'}
    )

In [ ]:
#| output: false
#| echo: false
fig_plotly.write_json("people_vote.json")
fig_plotly.write_image("featured.png")

Les cercles proportionnels permettent ainsi à l'oeil de se concentrer sur les 
zones les plus denses et non sur les grands espaces.

# Explorer la structure des données

La première étape nécessaire à suivre avant de se lancer dans la modélisation
est de déterminer les variables à inclure dans le modèle.

Les fonctionnalités de `pandas` sont, à ce niveau, suffisantes pour explorer des structures simples.
Néanmoins, lorsqu'on est face à un jeu de données présentant de
nombreuses variables explicatives (*features* en machine learning, *covariates* en économétrie),
il est souvent judicieux d'avoir une première étape de sélection de variables,
ce que nous verrons par la suite dans la [partie dédiée](https://linogaliana-teaching.netlify.app/lasso/).  

Avant d'être en mesure de sélectionner le meilleur ensemble de variables explicatives,
nous allons en prendre un nombre restreint et arbitraire.
La première tâche est de représenter les relations entre les données,
notamment la relation des variables explicatives
à la variable dépendante (le score du parti républicain)
ainsi que les relations entre les variables explicatives. 

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 2 : Regarder les corrélations entre les variables</h3>
```


1. Créer un DataFrame `df2` plus petit avec les variables `winner`, `votes_gop`, `Unemployment_rate_2019`,
`Median_Household_Income_2019`,
`Percent of adults with less than a high school diploma, 2015-19`,
`Percent of adults with a bachelor's degree or higher, 2015-19`
2. Représenter grâce à un graphique la matrice de corrélation avec `heatmap` de `seaborn`.
3. Représenter une matrice de nuages de points des variables de la base `df2` avec `pd.plotting.scatter_matrix`
4. (optionnel) Refaire ces figures avec `plotly` qui offre également la possibilité de faire une matrice de corrélation. 


```{=html}
</div>
```

:::


In [ ]:
#| output: false
#| echo: false

# 1. Créer le data.frame df2.
df2 = votes.set_index("GEOID").loc[: , ["winner", "votes_gop",
          'Unemployment_rate_2019', 'Median_Household_Income_2019',
          'Percent of adults with less than a high school diploma, 2015-19',
          "Percent of adults with a bachelor's degree or higher, 2015-19"]]

In [ ]:
#| echo: false
#| warning: false

# 2. Matrice de corrélation graphique
g1 = sns.heatmap(df2.drop("winner", axis = 1).corr(), cmap='coolwarm', annot=True, fmt=".2f")

## Construction directement avec pandas également possible
g2 = df2.drop("winner", axis = 1).corr().style.background_gradient(cmap='coolwarm').format('{:.2f}')

La matrice construite avec `seaborn` (question 2) aura l'aspect suivant:


In [ ]:
#| echo: false

g1.figure.get_figure()

Alors que celle construite directement avec `corr` de `pandas`
ressemblera plutôt à ce tableau :


In [ ]:
#| echo: false

g2

Le nuage de point obtenu à l'issue de la question 3 ressemblera à :


In [ ]:
#| echo: false
#| warning: false

# 3. Matrice de nuages de points
ax = pd.plotting.scatter_matrix(df2, figsize = (15,15))

In [ ]:
#| echo: false
ax

Le résultat de la question 4 devrait, quant à lui,
ressembler au graphique suivant :


In [ ]:
#| echo: false
#| output: false
 
# 4. Matrice de corélation avec plotly
import plotly
import plotly.express as px
htmlsnip2 = px.scatter_matrix(df2)
htmlsnip2.update_traces(diagonal_visible=False)
#htmlsnip2

In [ ]:
# Pour inclusion dans le site web
htmlsnip2.write_json("scatter.json")

{{< chart data="scatter" >}}




# Transformer les données

Les différences d'échelle ou de distribution entre les variables peuvent 
diverger des hypothèses sous-jacentes dans les modèles.

Par exemple, dans le cadre
de la régression linéaire, les variables catégorielles ne sont pas traitées à la même
enseigne que les variables ayant valeur dans $\mathbb{R}$. Une variable
discrète (prenant un nombre fini de valeurs) devra être transformées en suite de
variables 0/1 par rapport à une modalité de référence pour être en adéquation
avec les hypothèses de la régression linéaire.
On appelle ce type de transformation
*one-hot encoding*, sur lequel nous reviendrons. Il s'agit d'une transformation,
parmi d'autres, disponibles dans `scikit` pour mettre en adéquation un jeu de
données et des hypothèses mathématiques. 

L'ensemble de ces tâches s'appelle le *preprocessing*. L'un des intérêts
d'utiliser `scikit` est qu'on peut considérer qu'une tâche de preprocessing
est une tâche d'apprentissage (on apprend des paramètres d'une structure 
de données) qui est réutilisable pour un jeu de données à la structure
similaire:

![](scikit_predict.png)


Nous allons voir deux processus très classiques de *preprocessing* : 

1. La **standardisation** transforme des données pour que la distribution empirique suive une loi $\mathcal{N}(0,1)$.

2. La **normalisation**  transforme les données de manière à obtenir une norme ($\mathcal{l}_1$ ou $\mathcal{l}_2$) unitaire. Autrement dit, avec la norme adéquate, la somme des éléments est égale à 1.

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-triangle-exclamation"></i> Warning</h3>
```

Pour un statisticien,
le terme `normalization` dans le vocable `scikit` peut avoir un sens contre-intuitif.
On s'attendrait à ce que la normalisation consiste à transformer une variable de manière à ce que $X \sim \mathcal{N}(0,1)$.
C'est, en fait, la **standardisation** en `scikit` qui fait cela.


```{=html}
</div>
```

:::


## Standardisation

La standardisation consiste à transformer des données pour que la distribution empirique suive une loi $\mathcal{N}(0,1)$. Pour être performants, la plupart des modèles de machine learning nécessitent souvent d'avoir des données dans cette distribution.

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 3: Standardisation</h3>
```



1. Standardiser la variable `Median_Household_Income_2019` (ne pas écraser les valeurs !) et regarder l'histogramme avant/après normalisation.

*Note : On obtient bien une distribution centrée à zéro et on pourrait vérifier que la variance empirique soit bien égale à 1. On pourrait aussi vérifier que ceci est vrai également quand on transforme plusieurs colonnes à la fois.*


2. Créer `scaler`, un `Transformer` que vous construisez sur les 1000 premières lignes de votre DataFrame `df2`  à  l'exception de la variable à expliquer `winner`. Vérifier la moyenne et l'écart-type de chaque colonne sur ces mêmes observations.

*Note : Les paramètres qui seront utilisés pour une standardisation ultérieure sont stockés dans les attributs `.mean_` et `.scale_`*

On peut voir ces attributs comme des paramètres entraînés sur un certain jeu de
données et qu'on peut réutiliser sur un autre, à condition que les
dimensions coïncident.

3. Appliquer `scaler` sur les autres lignes du DataFrame et comparer les distributions obtenues de la variable `Median_Household_Income_2019`.

*Note : Une fois appliqués à un autre `DataFrame`, on peut remarquer que la distribution n'est pas exactement centrée-réduite dans le `DataFrame` sur lequel les paramètres n'ont pas été estimés. C'est normal, l'échantillon initial n'était pas aléatoire, les moyennes et variances de cet échantillon n'ont pas de raison de coïncider avec les moments de l'échantillon complet.*



```{=html}
</div>
```

:::


In [ ]:
#| output: false
#| echo: false

# 1. Standardisation de Median_Household_Income_2019 et histogramme
from sklearn import preprocessing
df2['y_standard'] = preprocessing.scale(df2['Median_Household_Income_2019'])
f, axes = plt.subplots(2, figsize=(10, 10))
sns.distplot(df2["Median_Household_Income_2019"] , color="skyblue", ax=axes[0])
sns.distplot(df2["y_standard"] , color="olive", ax=axes[1])
#print(df2['y_standard'].mean())
#print(df2['y_standard'].var())

In [ ]:
#| echo: false

#plt.savefig('standardisation.png', bbox_inches='tight')
# ax

In [ ]:
#| include: false
#| echo: false

# 2. Créer un scaler
df2 = df2.drop("winner", axis = 1)
print("Moyenne de chaque variable sur 1000 premières observations avant : ", np.array(df2.head(1000).mean(axis=0)))
print("Ecart-type de chaque variable sur 1000 premières observations avant : ", np.array(df2.head(1000).std(axis=0)))
scaler = preprocessing.StandardScaler().fit(df2.head(1000))
scaler.transform(df2.head(1000))
print("Moyenne de chaque variable sur 1000 premières observations après : ", scaler.transform(df2.head(1000)).mean(axis=0))
print("Ecart-type de chaque variable sur 1000 premières observations après : ", scaler.transform(df2.head(1000)).std(axis=0))
#print(scaler.mean_)
#print(scaler.scale_)

In [ ]:
#| include: false
#| echo: false

# 3. Appliquer le scaler à toutes les autres lignes
X1 = scaler.transform(df2.head(1000))
X2 = scaler.transform(df2[1000:])
col_pos = df2.columns.get_loc("Median_Household_Income_2019")
# X2.mean(axis = 0)
# X2.std(axis = 0)
f, axes = plt.subplots(2, figsize=(10, 10))
sns.distplot(X1[:,col_pos] , color="skyblue", ax=axes[0])
sns.distplot(X2[:,col_pos] , color="olive", ax=axes[1])

In [ ]:
#| echo: false
#plt.savefig('standardisation2.png', bbox_inches='tight')
#axes

## Normalisation

La **normalisation** est l'action de transformer les données de manière
à obtenir une norme ($\mathcal{l}_1$ ou $\mathcal{l}_2$) unitaire.
Autrement dit, avec la norme adéquate, la somme des éléments est égale à 1.
Par défaut, la norme est dans $\mathcal{l}_2$.
Cette transformation est particulièrement utilisée en classification de texte ou pour effectuer du *clustering*.

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 4 : Normalisation</h3>
```


1. Normaliser la variable `Median_Household_Income_2019` (ne pas écraser les valeurs !) et regarder l'histogramme avant/après normalisation.
2. Vérifier que la norme $\mathcal{l}_2$ est bien égale à 1.


```{=html}
</div>
```

:::


In [ ]:
#| output: false
#| echo: false

# 1. Normalisation de Median_Household_Income_2019 et histogrammes
scaler = preprocessing.Normalizer().fit(df2.dropna(how = "any").head(1000))
X1 = scaler.transform(df2.dropna(how = "any").head(1000))

f, axes = plt.subplots(2, figsize=(10, 10))
sns.distplot(df2["Median_Household_Income_2019"] , color="skyblue", ax=axes[0])
sns.distplot(X1[:,col_pos] , color="olive", ax=axes[1])

In [ ]:
#| echo: false
#plt.savefig('normalisation.png', bbox_inches='tight')
# axes

In [ ]:
#| output: false
#| echo: false

# 2. Vérification de la norme L2
np.sqrt(np.sum(X1**2, axis=1))[:10] # L2-norm

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-triangle-exclamation"></i> Warning</h3>
```

`preprocessing.Normalizer` n'accepte pas les valeurs manquantes, alors que `preprocessing.StandardScaler()` s'en accomode (dans la version `0.22` de scikit). Pour pouvoir aisément appliquer le *normalizer*, il faut

* retirer les valeurs manquantes du DataFrame avec la méthode `dropna`: `df.dropna(how = "any")`;
* ou les imputer avec un modèle adéquat. [`scikit` permet de le faire](https://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).


```{=html}
</div>
```

:::


## Encodage des valeurs catégorielles

Les données catégorielles doivent être recodées
sous forme de valeurs numériques pour être intégrés aux modèles de *machine learning*.
Cela peut être fait de plusieurs manières :

* `LabelEncoder`: transforme un vecteur `["a","b","c"]` en vecteur numérique `[0,1,2]`.
Cette approche a l'inconvénient d'introduire un ordre dans les modalités, ce qui n'est pas toujours souhaitable

* `OrdinalEncoder`: une version généralisée du `LabelEncoder` qui a vocation à s'appliquer sur des matrices ($X$),
alors que `LabelEncoder` s'applique plutôt à un vecteur ($y$)

* `pandas.get_dummies` effectue une opération de *dummy expansion*.
Un vecteur de taille *n* avec *K* catégories sera transformé en matrice de taille $n \times K$
pour lequel chaque colonne sera une variable *dummy* pour la modalité *k*.
Il y a ici $K$ modalités et il y a donc multicolinéarité.
Avec une régression linéaire avec constante,
il convient de retirer une modalité avant l'estimation.

* `OneHotEncoder` est une version généralisée (et optimisée) de la *dummy expansion*.
Il a plutôt vocation à s'appliquer sur les *features* ($X$) du modèle



::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 5 : Encoder des variables catégorielles</h3>
```


1. Créer `df` qui conserve uniquement les variables `state_name` et `county_name` dans `votes`.
2. Appliquer à `state_name` un `LabelEncoder`
*Note : Le résultat du label encoding est relativement intuitif, notamment quand on le met en relation avec le vecteur initial.*

3. Regarder la *dummy expansion* de `state_name`
4. Appliquer un `OrdinalEncoder` à `df[['state_name', 'county_name']]`
*Note : Le résultat du _ordinal encoding_ est cohérent avec celui du label encoding*

5. Appliquer un `OneHotEncoder` à `df[['state_name', 'county_name']]`

*Note : `scikit` optimise l'objet nécessaire pour stocker le résultat d'un modèle de transformation. Par exemple, le résultat de l'encoding One Hot est un objet très volumineux. Dans ce cas, `scikit` utilise une matrice Sparse.*



```{=html}
</div>
```

:::


In [ ]:
#| include: false
#| echo: false

#1. Création de df
df = votes[["state_name",'county_name']]

In [ ]:
#| include: false
#| echo: false

#2. Appliquer un LabelEncoder à stat_name
label_enc = preprocessing.LabelEncoder().fit(df['state_name'])
np.column_stack((label_enc.transform(df['state_name']),df['state_name']))

In [ ]:
#| include: false
#| echo: false

# 3. dummy expansion de state_name
pd.get_dummies(df['state_name'])

In [ ]:
#| include: false
#| echo: false

# 4. OrdinalEncoder
ord_enc = preprocessing.OrdinalEncoder().fit(df)
# ord_enc.transform(df[['state', 'county']])
ord_enc.transform(df)[:,0]

In [ ]:
#| include: false
#| echo: false

# 5. OneHotEncoder
onehot_enc = preprocessing.OneHotEncoder().fit(df)
onehot_enc.transform(df)

## Références

::: {#refs}
:::